## Chapter 9: Data Cleansing

Data cleansing is the process of detecting, correcting, and removing errors and inconsistences from data. Such errors may be the result of bad user input, which may include incorrect spelling, numeric entry errors, inconsistent abbreviation of names and addresses, and so on. Likewise, some errors may occur due to a faulty sensor on an Internet of Things (IoT) device or a noisy data transmission line. Regardless of the cause of the error, the goal of data cleansing is higher quality data.

This notebook performs operations on a database which has been pre-built for you. The commands used to build this database can be found in the chapter09-database.sql script file located in the chapter folder.

*Note: None of the MySQL statements found in this notebook are valid in the Jupyter environment. You are to run the code in the MySQL Workbench instance you prepared in *Getting Started with MySQL Workbench.*

# Testing for Non-Existent Records, Fields, and Null Values

Completeness is a measure of the degree to which the data represents all required values. To measure the completeness of your data, you can determine first what percentage of the expected data values are present within your data set. For example, if a process collects a sensor’s values every second, for each hour of data, you should have 3,600 records:
```python
(60 records/minute)*(60 minutes/hour) = 3,600 records/hour
```
For simplicity, assume you record and store one minute of data (60 seconds). You can compare the number of records you have to the number of records you expect. 

First, connect to the pre-built CHAPTER09 database by issuing the following command:

In [ ]:
USE CHAPTER09

Then, determine the completeness of the Sensor data by counting the number of records and dividing that number by 60. If the returned value is greater than 1, this is an indicator your table has duplicate records. If less than 1, this indicates missing records.

In [ ]:
############################
# Chapter 9 / Deliverable 1
############################

SELECT COUNT(*)/62 AS ‘Completeness’ FROM Sensor

In this case, the Completeness value is greater than 1—an indicator that your table has duplicate records. Later in this lab, you will learn how to quickly identify the duplicate records.

Next, for each field within every record, you will want to determine the percentage of fields that do not have values:

In [ ]:
SELECT COUNT(*) FROM Sensor WHERE SensorValue IS NULL

If you find that NULL values exist, you can display those records by simply replacing ```COUNT(*)``` with ```*```, indicating that you would just like to SELECT all records, rather than SELECT them and perform a count on them.

In [ ]:
############################
# Chapter 9 / Deliverable 2
############################

SELECT COUNT(*) FROM Sensor WHERE SensorValue IS NULL

# Testing for Duplicate Values

Depending on the size of your dataset, quickly determining if duplicate records exists can seem like a challenging process—you likely cannot determine the duplicate records at a glance. To determine if the data set contains duplicate records, first determine the number of records the data set contains:

In [ ]:
SELECT COUNT(*) AS 'Actual Record Count' FROM Customers

Next, determine the number of DISTINCT records in the set:

In [ ]:
SELECT COUNT(*) AS 'Distinct Record Count' FROM (Select DISTINCT * FROM Customers) c

If the two queries return the same count, the data set does not have duplicate records. If the first query contains more records than the second, duplicate records exist.

Next, to determine which records have more than the expected number, you would select all customers, count the number of records, group the customers by field, and then filter for records that appeared more than 1 time:

In [ ]:
SELECT *, COUNT(*) FROM Customers
GROUP BY CustomerID, FirstName, LastName
HAVING COUNT(*) > 1

In this case, by grouping on three key fields, we have a very high probability that the records are the same. To be exact, you can group on all the fields. If you examine the Customers table, you would find the duplicate records. This query, however, would not detect a customer who is in the system twice, but with different customer IDs. To detect those customers, you would issue the following query that groups records on each field except the CustomerID:

In [ ]:
SELECT *, COUNT(*) FROM Customers
GROUP BY FirstName, LastName, Age, Address, City, State, Zip, Birthday
HAVING COUNT(*) > 1

A similar operation would be performed to search for duplicate records in the Sensor table. And so we would begin:

In [ ]:
############################
# Chapter 9 / Deliverable 3
############################

SELECT Record, SensorValue, COUNT(*) FROM Sensor
GROUP BY Record, SensorValue

# Testing Value-Range Compliance

Accuracy is a measure of the degree to which the data correctly represents the corresponding real-world values. Assume, for example, that a sensor should only produce values in the range 0 to 100. To begin your accuracy measurement, you should test to insure that all the data values are in the range:

In [ ]:
SELECT COUNT(SensorValue) AS 'Errant Value Count' FROM Sensor 
WHERE SensorValue < 0 OR SensorValue > 100

To display the actual values that fall outside of the expected value range, issue the following query:

In [ ]:
SELECT Record, SensorValue FROM Sensor 
WHERE SensorValue < 0 OR SensorValue > 100

# Examining a Field’s Mean and Standard Deviation

As you evaluate the quality of your data, you may want to perform similar processing to identify potentially errant data. Furthermore, you may want to perform simple statistics on a field’s value to determine if you should further examine the field values for errors and inconsistencies. For example, assume your data contains records for a monitor device. Under normal operations, the device should return values in the range 50 to 60. Using the SQL AVG (average) and STDDEV (standard deviation) functions, you can gain insight into the field’s values. 

For a device returning values in the range 50 to 60, you would expect a small standard deviation and a mean value close to 55. Should the standard deviation become large, you would likely want to perform further exploration of the field values. 

In [ ]:
SELECT AVG(SensorValue), STDDEV(SensorValue) FROM Sensor

In this case, the large standard deviation is an indication of outlier values. 

Using the AVG and STDDEV functions, you can identify specific records that have values that do not necessarily align with others. For example, the following query will identify values that differ from the average by more than the twice standard deviation:

In [ ]:
############################
# Chapter 9 / Deliverable 4
############################

SELECT Record, SensorValue FROM Sensor
WHERE 
SensorValue > (SELECT AVG(SensorValue)+2*STDDEV(SensorValue) FROM Sensor) OR
SensorValue < (SELECT AVG(SensorValue)-2*STDDEV(SensorValue) FROM Sensor)

The query tests the SensorValue against values that are 2 standard deviations above the average and 2 standard deviations less than the average. By examining values that are greater than twice the standard deviation from the mean, you are likely to identify only data outliers. 

# Identifying Data Inconsistences

Consistency is a measure of the degree to which similar or related data values align throughout the data set. For systems that store user-input values, data consistency is always a challenge. For example, assume that a call-center user must enter a user’s address:

    123 North Main Street

The user might enter any of the following to represent the address such as:

    123 N Main Street
	123 N. Main Street
	123 No. Main Street
	123 North Main Street
	123 North Main St.
	123 North Main St
	123 N. Main St.
	123 N Main St.
	123 N Main St
	123 N. Main St

Although a mailperson will deliver mail to any of the listed addresses and will consider each address as the same, an SQL query will not.

Sticking with addresses, one user might specify a five-digit zip code (86305), whereas another user may input a nine-digit zip (86305-0001). Again, the data is conceptually the same, but not to an SQL query. 
Database developers refer to the process of correcting such data inconsistences as normalizing the data. Do not confuse this type of field normalization with the process of decomposing a relational table into first, second, or third normal form, as you may already be familiar with.
As you might guess, because they are often entered by users, addresses are constantly a data-consistency challenge. To start, you might test to confirm that each address uses the same zip code:

In [ ]:
SELECT A.Address, A.Zip, B.Zip FROM Customers A
INNER JOIN (SELECT Address, Zip FROM Customers) B
ON A.Address = B.Address WHERE A.Zip != B.Zip 
GROUP BY A.Address

The query performs a self-join (an inner join on itself, based on the address field). Then, it compares the zip codes of the joined records, returning those that differ. As you can see, the query uses table aliases to specify fields. 

# Eliminating Leading and Trailing Spaces
When a user enters data into a form, which a database later stores, there may be times when the user errantly types leading or trailing spaces before or after the values they type. Ideally, the application program storing the data will recognize and eliminate these extra characters. That said, as part of your data-cleansing process, you may need to eliminate them. In such cases, you can leverage the SQL RTRIM and LTRIM functions to remove the spaces:

In [ ]:
UPDATE Customers SET Firstname=RTrim(LTrim(FirstName)),
Lastname=RTrim(LTrim(LastName)),
Address=RTrim(LTrim(Address)),
City=RTrim(LTrim(City)),
State=RTrim(LTrim(State)),
Zip=RTrim(LTrim(Zip))

In this case, the UPDATE query will assign the specified fields with the value each contains minus leading spaces, which are removed by LTRIM and without the trailing spaces which are removed by RTRIM. When you execute the query, SQL will display the number of records it updated.

# Creating Your Own Custom Data Wrangler in Python

Depending on the complexity of the data they must transform or cleanse, database developers can use a programming language, such as Python, to create their own custom data-wrangling application. In this section, you will use Python to perform simple data-wrangling operations. You won’t need to create a Python program, in this case, but rather, you can execute the program statements using the Python interpreter. The purpose of this section is not to make you a Python programmer, but rather, to give you a feel for Python’s data cleansing capabilities. 

In this context, the Python package pandas has been pre-installed for you. However, should you have been required to install this package on your own, you would issue the following command:

```python
! pip install --user pandas
```

To start, you would typically import the pandas library, as powerful Python software library used for data manipulation and analysis. You would do this using the following line:
```python
import pandas as pd
```

Next, you would direct Python to create a dataframe to hold your data and to load into it the contents of the NeedsWrangling.csv file:
```python
dataFile = pd.read_csv('NeedsWrangling.csv')
```

Once you have loaded your dataset into a dataframe object, you could begin performing actions on it. For example, using Python’s print method, you can display the number of rows and columns your data contains:
```python
print(dataFile.shape)
```

In [ ]:
############################
# Chapter 9 / Deliverable 6
############################

# import pandas here
# load NeedsWrangling.csv into a dataframe object
# Print the number of rows and columns in the dataframe

To display the first five rows in your data, use the Python head method:

In [ ]:
dataFile.head()

Note that the head method displays the first 5 rows by default. To specify the number of rows to display, you would pass an integer value to the method.

The NeedsWrangling.csv file contains duplicate records. To view the duplicated records, use the duplicated method. The method returns a list of Boolean values that specify whether a row is a duplicate. The example below returns the first 7 rows with their corresponding Boolean value:

In [ ]:
############################
# Chapter 9 / Deliverable 7
############################

dupList = dataFile.duplicated()
dupList.head(7)

To remove the duplicate records, use the drop_duplicates method and assign the result to the data set:

In [ ]:
dataFile = dataFile.drop_duplicates('CustomerID')
dataFile

Several of the fields within the NeedsWrangling.csv file have leading and trailing spaces. To eliminate such whitespace from a column, perform the following:

In [ ]:
dataFile['Firstname'] = dataFile['Firstname'].map(str.strip)

If you examine the State column, you will find that some records use an uppercase value, some lowercase, and possibly some mixed case. Maybe you want to capitalize all the values in the a column, in which case you could use the upper() method. Or perhaps want to capitalize the first letter of each word in a column, in which case you could use the capitalize() method. The following example useds upper() to capitalize all letters in the State column:

In [ ]:
############################
# Chapter 9 / Deliverable 8
############################

dataFile['State'] = dataFile['State'].map(lambda s : s.upper())
dataFile

As discussed earlier, user-entered addresses and zip codes often yield inconsistent data. One way to identify potential problems is to display a count of values. The following operation displays a count of the different zip codes:

In [ ]:
dataFile['Zip'].value_counts()

Often, as you analyze data, you will want to sort the data based on a column’s values. The following command directs Pandas to sort the data values by Lastname:

In [ ]:
dataFile.sort_values('Lastname')

Just as the following statement directs Pandas to display only the Address column:

In [ ]:
dataFile["Address"]

Finally, analysts often want to know if NULL or NaN (Not a Value) values exist. Using the isnull method, you can direct pandas to report whether or not a field is NULL or NaN. By default, Pandas will return a true or false value for every field. As an alternative, you can use the sum method to add up any such values:

In [ ]:
dataFile.isnull().sum()